In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9763634166055436116, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11542060566038745647
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [14]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=NASNetMobile(weights='imagenet', include_top=False, pooling = 'avg', input_tensor=Input(shape=input_shape))

In [15]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "NASNetMobile_descriptors"

In [16]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [17]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [18]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [19]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [20]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [21]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [22]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [23]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [24]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.8, 'lr': 1e-06, 'beta_2': 0.99}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.8383 - acc: 0.2067 - val_loss: 1.7479 - val_acc: 0.2060
Epoch 2/15
 - 4s - loss: 1.7816 - acc: 0.2499 - val_loss: 1.7387 - val_acc: 0.2158
Epoch 3/15
 - 4s - loss: 1.7260 - acc: 0.2963 - val_loss: 1.7296 - val_acc: 0.2456
Epoch 4/15
 - 4s - loss: 1.6842 - acc: 0.3323 - val_loss: 1.7196 - val_acc: 0.2714
Epoch 5/15
 - 4s - loss: 1.6440 - acc: 0.3739 - val_loss: 1.7091 - val_acc: 0.2889
Epoch 6/15
 - 4s - loss: 1.6076 - acc: 0.4030 - val_loss: 1.6977 - val_acc: 0.2986
Epoch 7/15
 - 4s - loss: 1.5727 - acc: 0.4358 - val_loss: 1.6859 - val_acc: 0.3069
Epoch 8/15
 - 4s - loss: 1.5470 - acc: 0.4561 - val_loss: 1.6736 - val_acc: 0.3115
Epoch 9/15
 - 4s - loss: 1.5153 - acc: 0.4742 - val_loss: 1.6610 - val_acc: 0.3151
Epoch 10/15
 - 4s - loss: 1.4903 - acc: 0.4962 - val_loss: 1.6486 - val_acc: 0.3178
Epoch 11/15
 - 4s - loss: 1.4632 - acc: 0.5092 - val_

Epoch 2/15
 - 4s - loss: 1.0221 - acc: 0.7001 - val_loss: 1.3834 - val_acc: 0.4511
Epoch 3/15
 - 4s - loss: 0.8862 - acc: 0.7223 - val_loss: 1.3356 - val_acc: 0.4610
Epoch 4/15
 - 4s - loss: 0.8054 - acc: 0.7387 - val_loss: 1.2887 - val_acc: 0.4753
Epoch 5/15
 - 4s - loss: 0.7514 - acc: 0.7514 - val_loss: 1.2552 - val_acc: 0.4920
Epoch 6/15
 - 4s - loss: 0.7118 - acc: 0.7594 - val_loss: 1.2333 - val_acc: 0.5043
Epoch 7/15
 - 4s - loss: 0.6795 - acc: 0.7687 - val_loss: 1.1942 - val_acc: 0.5219
Epoch 8/15
 - 4s - loss: 0.6509 - acc: 0.7765 - val_loss: 1.1816 - val_acc: 0.5330
Epoch 9/15
 - 4s - loss: 0.6289 - acc: 0.7822 - val_loss: 1.1624 - val_acc: 0.5440
Epoch 10/15
 - 4s - loss: 0.6120 - acc: 0.7893 - val_loss: 1.1617 - val_acc: 0.5482
Epoch 11/15
 - 4s - loss: 0.5963 - acc: 0.7915 - val_loss: 1.1273 - val_acc: 0.5599
Epoch 12/15
 - 4s - loss: 0.5786 - acc: 0.7981 - val_loss: 1.1320 - val_acc: 0.5634
Epoch 13/15
 - 4s - loss: 0.5677 - acc: 0.8021 - val_loss: 1.1176 - val_acc: 0.5701


Epoch 9/15
 - 4s - loss: 0.5495 - acc: 0.8369 - val_loss: 1.4038 - val_acc: 0.6383
Epoch 10/15
 - 4s - loss: 0.5562 - acc: 0.8357 - val_loss: 1.3868 - val_acc: 0.6498
Epoch 11/15
 - 4s - loss: 0.5712 - acc: 0.8348 - val_loss: 1.3665 - val_acc: 0.6612
Epoch 12/15
 - 4s - loss: 0.5834 - acc: 0.8369 - val_loss: 1.4275 - val_acc: 0.6512
Epoch 13/15
 - 4s - loss: 0.5997 - acc: 0.8337 - val_loss: 1.4993 - val_acc: 0.6524
Epoch 14/15
 - 4s - loss: 0.5920 - acc: 0.8359 - val_loss: 1.4899 - val_acc: 0.6549
Epoch 00014: early stopping
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 65.4900%
Validation Loss: 1.489928001469533
Test Accuracy: 0.8032110767417882
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'lr': 1e-07, 'beta_2': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.8331 - acc: 0.200

 - 4s - loss: 0.8329 - acc: 0.7122 - val_loss: 1.2749 - val_acc: 0.4598
Epoch 14/15
 - 4s - loss: 0.8177 - acc: 0.7167 - val_loss: 1.2633 - val_acc: 0.4678
Epoch 15/15
 - 4s - loss: 0.8014 - acc: 0.7210 - val_loss: 1.2515 - val_acc: 0.4761
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 47.6133%
Validation Loss: 1.251471195091521
Test Accuracy: 0.6677713949035204
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.6, 'lr': 0.1, 'beta_2': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 3.1935 - acc: 0.4807 - val_loss: 2.6255 - val_acc: 0.4085
Epoch 2/15
 - 4s - loss: 2.4440 - acc: 0.5276 - val_loss: 3.7371 - val_acc: 0.5166
Epoch 3/15
 - 4s - loss: 2.5741 - acc: 0.5576 - val_loss: 5.1247 - val_acc: 0.5188
Epoch 4/15
 - 4s - loss: 2.4592 - acc: 0.5647 - val_loss: 3.3990 - val_acc: 0.5432
Epoch

 - 4s - loss: 0.5062 - acc: 0.8215 - val_loss: 0.9929 - val_acc: 0.6092
Epoch 10/15
 - 4s - loss: 0.4937 - acc: 0.8249 - val_loss: 0.9815 - val_acc: 0.6153
Epoch 11/15
 - 4s - loss: 0.4818 - acc: 0.8289 - val_loss: 0.9905 - val_acc: 0.6143
Epoch 12/15
 - 4s - loss: 0.4698 - acc: 0.8329 - val_loss: 0.9786 - val_acc: 0.6214
Epoch 13/15
 - 4s - loss: 0.4608 - acc: 0.8373 - val_loss: 0.9572 - val_acc: 0.6267
Epoch 14/15
 - 4s - loss: 0.4510 - acc: 0.8406 - val_loss: 0.9636 - val_acc: 0.6302
Epoch 15/15
 - 4s - loss: 0.4464 - acc: 0.8420 - val_loss: 0.9482 - val_acc: 0.6364
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 63.6382%
Validation Loss: 0.9482210941751192
Test Accuracy: 0.7867874502872293
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.0001, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 sam

...

Epoch 7/15
 - 10s - loss: 1.8130 - acc: 0.2041 - val_loss: 1.7759 - val_acc: 0.2393
Epoch 8/15
 - 10s - loss: 1.8044 - acc: 0.2119 - val_loss: 1.7740 - val_acc: 0.2495
Epoch 9/15
 - 10s - loss: 1.7968 - acc: 0.2179 - val_loss: 1.7722 - val_acc: 0.2583
Epoch 10/15
 - 9s - loss: 1.7953 - acc: 0.2228 - val_loss: 1.7704 - val_acc: 0.2684
Epoch 11/15
 - 10s - loss: 1.7871 - acc: 0.2301 - val_loss: 1.7686 - val_acc: 0.2758
Epoch 12/15
 - 9s - loss: 1.7789 - acc: 0.2372 - val_loss: 1.7669 - val_acc: 0.2825
Epoch 13/15
 - 12s - loss: 1.7765 - acc: 0.2413 - val_loss: 1.7651 - val_acc: 0.2881
Epoch 14/15
 - 11s - loss: 1.7710 - acc: 0.2477 - val_loss: 1.7634 - val_acc: 0.2926
Epoch 15/15
 - 10s - loss: 1.7670 - acc: 0.2516 - val_loss: 1.7617 - val_acc: 0.2979
14958/14958 [==============================] - 2s 122us/step
Validation Accuracy: 29.7901%
Validation Loss: 1.7616854717362005
Test Accuracy: 0.36404477831786713
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

Epoch 5/15
 - 11s - loss: 0.6247 - acc: 0.7874 - val_loss: 1.1417 - val_acc: 0.5777
Epoch 6/15
 - 11s - loss: 0.6036 - acc: 0.7957 - val_loss: 1.1113 - val_acc: 0.5889
Epoch 7/15
 - 10s - loss: 0.5911 - acc: 0.8031 - val_loss: 1.1195 - val_acc: 0.5975
Epoch 8/15
 - 10s - loss: 0.5882 - acc: 0.8052 - val_loss: 1.1157 - val_acc: 0.5969
Epoch 9/15
 - 10s - loss: 0.5762 - acc: 0.8119 - val_loss: 1.0796 - val_acc: 0.6084
Epoch 10/15
 - 10s - loss: 0.5669 - acc: 0.8151 - val_loss: 1.1109 - val_acc: 0.6096
Epoch 11/15
 - 11s - loss: 0.5665 - acc: 0.8185 - val_loss: 1.1326 - val_acc: 0.6133
Epoch 12/15
 - 10s - loss: 0.5671 - acc: 0.8204 - val_loss: 1.1196 - val_acc: 0.6177
Epoch 13/15
 - 10s - loss: 0.5604 - acc: 0.8228 - val_loss: 1.1225 - val_acc: 0.6231
Epoch 14/15
 - 10s - loss: 0.5647 - acc: 0.8254 - val_loss: 1.1282 - val_acc: 0.6250
Epoch 15/15
 - 12s - loss: 0.5595 - acc: 0.8294 - val_loss: 1.1530 - val_acc: 0.6249
14958/14958 [==============================] - 2s 161us/step
Validatio

Epoch 11/15
 - 11s - loss: 1.8013 - acc: 0.2125 - val_loss: 1.7602 - val_acc: 0.3226
Epoch 12/15
 - 10s - loss: 1.7943 - acc: 0.2211 - val_loss: 1.7582 - val_acc: 0.3304
Epoch 13/15
 - 9s - loss: 1.7881 - acc: 0.2303 - val_loss: 1.7563 - val_acc: 0.3377
Epoch 14/15
 - 10s - loss: 1.7844 - acc: 0.2290 - val_loss: 1.7543 - val_acc: 0.3447
Epoch 15/15
 - 13s - loss: 1.7770 - acc: 0.2376 - val_loss: 1.7524 - val_acc: 0.3513
14958/14958 [==============================] - 2s 129us/step
Validation Accuracy: 35.1250%
Validation Loss: 1.752395824867067
Test Accuracy: 0.38297245544262776
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.7067 - acc: 0.7511 - val_loss: 1.0605 - val_acc: 0.5762
Epoch 2/15
 - 11s - loss: 0.5213 - acc: 0.8151 - val_loss: 0.99

Epoch 2/15
 - 10s - loss: 0.7682 - acc: 0.7343 - val_loss: 1.1741 - val_acc: 0.5185
Epoch 3/15
 - 10s - loss: 0.6736 - acc: 0.7660 - val_loss: 1.1075 - val_acc: 0.5538
Epoch 4/15
 - 10s - loss: 0.6208 - acc: 0.7829 - val_loss: 1.0585 - val_acc: 0.5721
Epoch 5/15
 - 10s - loss: 0.5853 - acc: 0.7926 - val_loss: 1.0518 - val_acc: 0.5797
Epoch 6/15
 - 10s - loss: 0.5576 - acc: 0.8029 - val_loss: 1.0239 - val_acc: 0.5924
Epoch 7/15
 - 9s - loss: 0.5372 - acc: 0.8102 - val_loss: 1.0096 - val_acc: 0.6005
Epoch 8/15
 - 10s - loss: 0.5196 - acc: 0.8149 - val_loss: 1.0061 - val_acc: 0.5990
Epoch 9/15
 - 11s - loss: 0.5045 - acc: 0.8221 - val_loss: 1.0049 - val_acc: 0.6077
Epoch 10/15
 - 11s - loss: 0.4913 - acc: 0.8262 - val_loss: 0.9956 - val_acc: 0.6118
Epoch 11/15
 - 10s - loss: 0.4780 - acc: 0.8312 - val_loss: 0.9968 - val_acc: 0.6128
Epoch 12/15
 - 11s - loss: 0.4707 - acc: 0.8342 - val_loss: 0.9687 - val_acc: 0.6227
Epoch 13/15
 - 10s - loss: 0.4606 - acc: 0.8359 - val_loss: 0.9716 - val_a

Parameters testing:  {'beta_1': 0.6, 'lr': 0.1, 'beta_2': 0.995}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 5.5618 - acc: 0.4586 - val_loss: 4.8918 - val_acc: 0.2695
Epoch 2/15
 - 11s - loss: 4.5424 - acc: 0.4573 - val_loss: 4.1086 - val_acc: 0.3441
Epoch 3/15
 - 9s - loss: 3.6122 - acc: 0.4404 - val_loss: 2.6763 - val_acc: 0.3567
Epoch 4/15
 - 10s - loss: 3.0639 - acc: 0.4745 - val_loss: 3.7116 - val_acc: 0.2622
Epoch 5/15
 - 10s - loss: 2.3399 - acc: 0.4684 - val_loss: 3.4804 - val_acc: 0.4198
Epoch 6/15
 - 8s - loss: 1.7918 - acc: 0.4568 - val_loss: 1.8010 - val_acc: 0.3897
Epoch 7/15
 - 10s - loss: 1.7626 - acc: 0.4632 - val_loss: 2.8432 - val_acc: 0.4571
Epoch 8/15
 - 10s - loss: 1.9443 - acc: 0.4801 - val_loss: 2.0843 - val_acc: 0.3987
Epoch 9/15
 - 10s - loss: 1.9873 - acc: 0.4668 - val_loss: 2.6525 - val_acc: 0.4495
Epoch 10/15
 - 10s - loss: 1.8894 - acc: 0.4726 - val_loss: 2.4711 - val_acc: 0.3829
Epoch 00010: early stopping
14958/14958 [=====